In [ ]:
import pydsm
from pydsm import hydroh5
import h5py
import numpy as np
import pandas as pd

In [ ]:
import shapely
import geopandas as gpd
import hvplot.pandas
import holoviews as hv
from holoviews import opts
import geoviews as gv
hv.extension('bokeh')
import panel as pn
pn.extension()
import colorcet as cc

In [ ]:
print('Holoviews version: ',hv.__version__)
print('Geopandas version: ',gpd.__version__)
print('Shapely version: ',shapely.__version__)

In [ ]:
#file='../tests/data/historical_v82_ec.h5'
#file='d:/delta/DSM2v821/study_templates/historical/output/hist_v821_EC.h5'
file='D:/dev/pydsm/tests/data/gtm_sample_output/historical_gtm.h5'

In [ ]:
def load_df(file, offset=0, chunk_size=5000, constituent_id=0):
    qh5=h5py.File(file,'r')
    #qh5.visit(lambda name: print(name))
    tcnames=qh5.get('output/constituent_names')
    df_constituent_names=pd.DataFrame(tcnames)
    df_constituent_names[constituent_id]=df_constituent_names[constituent_id].str.decode('utf-8')
    df_constituent_names[constituent_id].values
    #df_constituent_names[0][0].decode('UTF-8')
    tsdata=qh5.get('output/channel concentration')
    ttsdata=tsdata[offset:(offset+chunk_size),constituent_id,:,:]
    attrs={k: tsdata.attrs[k] for k in tsdata.attrs.keys()}
    #ttsdata.shape
    #dfts=pd.DataFrame(data=ttsdata[])
    #ttsdata.mean(axis=2).shape
    data_len=ttsdata.shape[0]
    nchans=ttsdata.shape[1]
    nloc=ttsdata.shape[2]
    mi=pd.MultiIndex.from_product([chan_index,['u','d']],names=['channel_number','u/d'])
    start_time=pd.to_datetime(attrs['start_time'][0].decode('UTF-8'))
    time_interval=pd.to_timedelta(attrs['interval'][0].decode('UTF-8'))
    time_index=pd.date_range(start=start_time,periods=ttsdata.shape[0],freq=time_interval)
    #index=start_time+np.arange(0,ttsdata.shape[0])*time_intervals
    #print(index+np.arange(0,ttsdata.shape[0])*'H')
    print(start_time)
    ttsdf=pd.DataFrame(ttsdata.reshape(data_len,nchans*nloc),index=time_index,columns=mi)
    return ttsdata,ttsdf,attrs,start_time,time_interval

In [ ]:
def get_channel_numbers(file):
    qh5=h5py.File(file,'r')
    return pd.DataFrame(qh5.get('output/channel_number'))[0].values
chan_index=get_channel_numbers(file)

In [ ]:
ttsdata,ttsdf,attrs,start_time,time_interval=load_df(file,chunk_size=50000)

In [ ]:
def godin_filter_array(data):
    '''godin filter the channel data for loaded time'''
    from vtools.functions import filter
    godin_fir=filter.generate_godin_fir(attrs['interval'][0].decode('UTF-8').replace('min','T'))
    for outer in range(data.shape[2]):
        for inner in range(data.shape[1]):
            data[:,inner,outer]=np.convolve(data[:,inner, outer],godin_fir,mode='same')

In [ ]:
ttsdatag=godin_filter_array(ttsdata)

In [ ]:
data_len=ttsdata.shape[0]
nchans=ttsdata.shape[1]
nloc=ttsdata.shape[2]

In [ ]:
print(data_len,nchans,nloc)

In [ ]:
ttsdf_d=ttsdf.resample('D').mean()

In [ ]:
def get_data(ti=0):
    return pd.DataFrame(ttsdf.iloc[ti].values.reshape(nchans,nloc).mean(axis=1),chan_index,columns=['data'])

In [ ]:
dsm2_chans=gpd.read_file('../pydsm/maps/v8.2/DSM2_Flowline_Segments.shp').to_crs(epsg=3857)
#dsm2_chans.geometry=dsm2_chans.geometry.simplify(tolerance=150)
dsm2_chans.geometry=dsm2_chans.geometry.buffer(250,cap_style=1, join_style=1)
#dsm2_chans.head()

In [ ]:
df=get_data(0)
dsm2_chans_m=dsm2_chans.merge(df,left_on='channel_nu',right_index=True)

In [ ]:
map=hv.element.tiles.CartoLight().opts(width=800,height=600,alpha=0.5)
df=get_data(0)
cur_time = start_time+pd.to_timedelta(attrs['interval'][0].decode('UTF-8'))# ti*pd.to_timedelta('1D')#
tlabel = cur_time.strftime('%Y-%m-%d %H:%M')
dsm2_chans_m=dsm2_chans.merge(df,left_on='channel_nu',right_index=True)
chans_plot=dsm2_chans_m.hvplot(c='data',line_alpha=0, alpha=0.95, cmap='rainbow')
chans_plot=chans_plot.redim.range(data=(100,1000)).opts(framewise=False)
view=(map*chans_plot).opts(title='Time: %s'%tlabel,framewise=False)

In [ ]:
import time
from bokeh.io import push_notebook, show, output_notebook
rendered_view=hv.render(view)
pr=rendered_view.renderers[1]
target = show(rendered_view, notebook_handle=True)
def update_view(i=0):
    df=get_data(i)
    cur_time = start_time+i*pd.to_timedelta(attrs['interval'][0].decode('UTF-8'))# ti*pd.to_timedelta('1D')#
    tlabel = cur_time.strftime('%Y-%m-%d %H:%M')
    rendered_view.title.text=tlabel
    dsm2_chans_mi=dsm2_chans.merge(df,left_on='channel_nu',right_index=True)
    pr.data_source.data['color']=dsm2_chans_mi.data.values
    pr.data_source.data['data']=dsm2_chans_mi.data.values
    push_notebook(handle=target)
import panel as pn
pn.extension()
pn.interact(update_view,i=(0,50000))

In [ ]:
import time
#from bokeh.io import push_notebook, show, output_notebook
#rendered_view=hv.render(view)
#pr=rendered_view.renderers[1]
#target = show(rendered_view, notebook_handle=True)
map=hv.element.tiles.CartoLight().opts(width=800,height=600,alpha=0.5)
def update_view(i=0):
    df=get_data(i)
    cur_time = start_time+i*pd.to_timedelta(attrs['interval'][0].decode('UTF-8'))# ti*pd.to_timedelta('1D')#
    tlabel = cur_time.strftime('%Y-%m-%d %H:%M')
    rendered_view.title.text=tlabel
    dsm2_chans_mi=dsm2_chans.merge(df,left_on='channel_nu',right_index=True)
df=get_data(0)
cur_time = start_time+pd.to_timedelta(attrs['interval'][0].decode('UTF-8'))# ti*pd.to_timedelta('1D')#
tlabel = cur_time.strftime('%Y-%m-%d %H:%M')
dsm2_chans_m=dsm2_chans.merge(df,left_on='channel_nu',right_index=True)
chans_plot=dsm2_chans_m.hvplot(c='data',line_alpha=0, alpha=0.95, cmap='rainbow')
chans_plot=chans_plot.redim.range(data=(100,1000)).opts(framewise=False)
view=(map*chans_plot).opts(title='Time: %s'%tlabel,framewise=False)
#import panel as pn
#pn.extension()
#pn.interact(update_view,i=(0,50000))

In [ ]:
import datashader as ds, pandas as pd
import datashader.transfer_functions as tf
from holoviews.operation.datashader import datashade

In [ ]:
import panel as pn
pn.extension()
map=hv.element.tiles.CartoLight().opts(width=800,height=600,alpha=0.5)
def channel_map(ti):
    df=get_data(ti)
    cur_time = start_time+ti*pd.to_timedelta('1D')#pd.to_timedelta(attrs['interval'][0].decode('UTF-8'))
    tlabel = cur_time.strftime('%Y-%m-%d %H:%M')
    dsm2_chans_m2=dsm2_chans.merge(df,left_on='channel_nu',right_index=True)
    dsm2_chans_m['data']=dsm2_chans_m2['data']
    chans_plot=hv.Polygons(dsm2_chans_m,vdims=['data']).opts(line_alpha=0, alpha=0.95, cmap=cc.rainbow)
    chans_plot=chans_plot.redim.range(data=(100,1000))#.opts(framewise=False)
    ov=map*chans_plot.opts(title='Time: %s'%tlabel)#,framewise=False)
    return ov
dmap=hv.DynamicMap(channel_map,kdims=['ti'])
anim_pane=dmap.redim.range(ti=(0,len(ttsdf_d)-1)).opts(framewise=False)
p=pn.panel(anim_pane)
titlePane=pn.pane.Markdown(''' # %s Animation (Tidally Filtered)'''%'EC')
anim_panel=pn.Column(pn.Row(titlePane,*p[1]),p[0])
pn.ipywidget(anim_panel)

In [ ]:
kw=dict(ti=(0,len(ttsdf_d)))
pn.interact(channel_map,**kw)

In [ ]:
import datetime as dt
date_slider = pn.widgets.DateSlider(name='Date Slider', start=dt.datetime(2019, 1, 1), end=dt.datetime(2019, 6, 1), value=dt.datetime(2019, 2, 8))
date_slider

In [ ]:
def norm_dist_for_ec(df1, ecx2=2700):
    '''
    Finds channels that containt the ecx2 value and then linearly interpolates to find normalized distance (0 -> 1)
    returns a data frame of norm_dist dataframe
    '''
    dfu=df1[:,'u']
    dfd=df1[:,'d']
    cond1=(dfu < ecx2) & (dfd > ecx2)
    cond2=(dfu > ecx2) & (dfd < ecx2)
    cond_between= cond1 | cond2
    #dfu[cond_between]
    #dfd[cond_between]
    return abs((ecx2-dfd[cond_between])/(ecx2-dfu[cond_between]))
    

In [ ]:
from scipy.interpolate import interp1d
from shapely.geometry import LineString

In [ ]:
dsm2_chanso=gpd.read_file('maps/v8.2/DSM2_Flowline_Segments.shp').to_crs(epsg=3857)

In [ ]:
def x2_map(ti,ecx2=2700):
    norm_dist=norm_dist_for_ec(ttsdf_d.iloc[ti],ecx2)
    dsm2_x2=dsm2_chans.join(norm_dist,on='channel_nu',how='right')
    #dsm2_x2
    x2_line_df=dsm2_x2.geometry.interpolate(dsm2_x2[dsm2_x2.columns[-1]])
    xl=x2_line_df.values.x
    yl=x2_line_df.values.y
    f=interp1d(x2_line_df.values.x, x2_line_df.values.y)
    x2_line=LineString(x2_line_df.values)
    return x2_line

In [ ]:
dmap[10]*hv.Path(x2_map(10)).opts(line_width=5, line_color='black')

In [ ]:
dmap[120]